# exchange

> This module interacts with exchange

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| default_exp exchange

In [ ]:
#| export
import imaplib
import email
from email.header import decode_header
from datetime import datetime
import os
import tldextract
from email.utils import parseaddr, getaddresses

Run the following in exchange to export your exchange user name and password (restart Terminal and Jupyter lab if it does not work):

```
echo 'export EXCHANGE_USER="YOUR_USERNAME"' >> ~/.bashrc && grep -q 'source ~/.bashrc' ~/.bash_profile || echo -e "\nif [ -f ~/.bashrc ]; then\n  source ~/.bashrc\nfi" >> ~/.bash_profile && source ~/.bashrc
```
and

```
echo 'export EXCHANGE_PASSWORD="YOUR_PASSWORD"' >> ~/.bashrc && grep -q 'source ~/.bashrc' ~/.bash_profile || echo -e "\nif [ -f ~/.bashrc ]; then\n  source ~/.bashrc\nfi" >> ~/.bash_profile && source ~/.bashrc
```

In [ ]:
#| export

def extract_domains(addresses):
    return [extract_domain(a) for a in addresses]

def extract_domain(address):
    domain = address.split('@')[1]
    # Extract main domain from subdomain
    main_domain = tldextract.extract(domain)
    return f"{main_domain.domain}.{main_domain.suffix}"

class EmailObject:
    def __init__(self, sender, recipient, cc, bcc, subject, body, email_id):
        self.sender = self.get_all_addresses(sender)
        self.recipient = self.get_all_addresses(recipient)
        self.cc = self.get_all_addresses(cc)
        self.bcc = self.get_all_addresses(bcc)
        self.subject = subject
        self.body = body
        self.email_id = email_id

    def get_all_addresses(self, tofrom):
        if not tofrom:
            return []
        addresses = [a[1] for a in getaddresses(tofrom.split(','))]
        addresses = [a for a in addresses if '@' in a]
        return addresses

class EmailClient:
    def __init__(self, server, port, username, password):
        self.mail = imaplib.IMAP4_SSL(server, port)
        self.mail.login(username, password)

    def get_emails(self, from_folder = 'inbox', since_date=None):
        self.mail.select(f'"{from_folder}"')
    
        if since_date:
            since_date_str = since_date.strftime('%d-%b-%Y')
            status, messages = self.mail.search(None, f'SINCE {since_date_str}')
        else:
            status, messages = self.mail.search(None, "ALL")
    
        email_ids = messages[0].split()
        email_objects = []
    
        for i in range(0, len(email_ids), 100):  # Fetch 100 emails at a time
            batch = email_ids[i:i+100]
            batch_str = ','.join(map(lambda x: x.decode(), batch))
            status, msg_data = self.mail.fetch(batch_str, "(RFC822)")
            index = 0
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])
                    current_id = batch[index]
                    index = index + 1
                    subject, encoding = decode_header(msg["Subject"])[0]
                    if isinstance(subject, bytes):
                        try:
                            subject = subject.decode(encoding if encoding else "utf-8")
                        except:
                            subject = ''
    
                    # Extract email text content only
                    body = ""
                    if msg.is_multipart():
                        for part in msg.walk():
                            if part.get_content_type() == "text/plain":
                                try:
                                    body = part.get_payload(decode=True).decode('utf-8')
                                except UnicodeDecodeError:
                                    body = part.get_payload(decode=True).decode('utf-8', 'ignore')
                                break  # Once text content is found, no need to continue
                    else:
                        try:
                            body = msg.get_payload(decode=True).decode('utf-8')
                        except UnicodeDecodeError:
                            body = msg.get_payload(decode=True).decode('utf-8', 'ignore')

    
                    # Create email object and add to list
                    email_objects.append(EmailObject(msg["From"], msg["To"], msg["Cc"], msg["Bcc"], subject, body, current_id))
    
        return email_objects

    def move_email(self, email_object, target_folder):
        email_id = email_object.email_id
        
        # Ensure email_id is a byte-string
        if not isinstance(email_id, bytes):
            email_id = email_id.encode('utf-8')
            
        result = self.mail.copy(email_id, target_folder)
        if result[0] == 'OK':
            self.mail.store(email_id, '+FLAGS', '(\Deleted)')
            self.mail.expunge()

    def logout(self):
        self.mail.logout()

In [ ]:
#| notest

USERNAME = os.environ.get("EXCHANGE_USER")
PASSWORD = os.environ.get("EXCHANGE_PASSWORD")

# Usage example
email_client = EmailClient("msx.tu-dresden.de", 993, USERNAME, PASSWORD)

# Fetch emails since a specific date
since_date = datetime(2023, 9, 21)  # Replace with actual date
emails = email_client.get_emails(from_folder = "inbox", since_date=since_date)

# Print and move emails
for e in emails:
    print(f"Subject: {e.subject}, ID: {e.bcc}")
    #email_client.move_email(e, 'Review')

# Logout to close the connection
email_client.logout()

Subject: AAT, ID: []
Subject: Hilmar, a recent article cited your research, ID: []
Subject: intense groups TRR, ID: []
Subject: WG: Interesse an einer Bachelorarbeit zum Projekt 1 (Zshg. PIT-Effekt und Tabakkonsumstörung), ID: []
Subject: Hilmar, we found new jobs you may be interested in, ID: []
Subject: [TRR265] , ID: []
Subject: Individuelle Prognose für Ihren Gewichtsverlust, ID: []
Subject: Graduation formalities, ID: []
Subject: Re: Question about 2017 Psychopharmacology study, ID: []
Subject: Congratulations Hilmar, you reached a milestone, ID: []
Subject: Test-retest reliability of a smartphone-based approach-avoidance task: Effects of retest period, stimulus type, and demographics., ID: []
Subject: Open Science Framework General: You are now unsubscribed, ID: []
Subject: Test-retest reliability of a smartphone-based approach-avoidance task: Effects of retest period, stimulus type, and demographics., ID: []
Subject: Regarding Article Contribution- Foot and Ankle, ID: []
Subject

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()